In [1]:
from satelite_data_preporation import collection
import ee 
import geemap
ee.Initialize()
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
begin = '2018-09-30'
end =  '2019-10-01'

In [4]:
two_fields = 'vector_data/archangelskoe_TWO_FIELDS_WGS84.geojson'
region_boundary = 'vector_data/budenovsk_district_boundaries.geojson'
region_of_interest = 'vector_data/our_fields_for_animation.geojson'
url = "https://rp5.ru/%D0%90%D1%80%D1%85%D0%B8%D0%B2_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D1%8B_%D0%B2_%D0%91%D1%83%D0%B4%D0%B5%D0%BD%D0%BD%D0%BE%D0%B2%D1%81%D0%BA%D0%B5"

In [5]:
region_boundary = 'vector_data/budenovsk_district_boundaries.geojson'
region_of_interest = 'vector_data/our_fields_for_animation.geojson'

sentinel2020_collection = collection( begin, end)
sentinel2020_collection.get_sattelit_collection('sentinel2',region_boundary, region_of_interest)
sentinel2020_collection.DownloadImages()
sentinel2020_collection.get_raster_plots(two_fields)
climat = sentinel2020_collection.anual_weather(url)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-01-10_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-02-19_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-03-21_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-03-26_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-03-31_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\

In [23]:
NDVI_anual = sentinel2020_collection.anual_ndvi(two_fields)

AttributeError: 'str' object has no attribute 'first_date'

In [22]:
NDVI_anual

,type,Dates,NDVI
0,PP,2018_09_30,0.2248
1,TT,2018_09_30,0.2156
2,PP,2018_10_16,0.2257
3,TT,2018_10_16,0.2139
4,PP,2018_11_01,0.2551
5,TT,2018_11_01,0.2285
6,PP,2018_11_17,0.2553
7,TT,2018_11_17,0.1527
8,PP,2018_12_03,0.3879
9,TT,2018_12_03,0.2052


,2018_09_30_NDVI,2018_10_16_NDVI,2018_11_01_NDVI,2018_11_17_NDVI,2018_12_03_NDVI,2018_12_19_NDVI,2019_01_01_NDVI,2019_01_17_NDVI,2019_02_02_NDVI,2019_02_18_NDVI,...,2019_06_10_NDVI,2019_06_26_NDVI,2019_07_12_NDVI,2019_07_28_NDVI,2019_08_13_NDVI,2019_08_29_NDVI,2019_09_14_NDVI,2019_09_30_NDVI,system:index,type
0,2248.0,2257.0,2551.0,2553.0,3879.0,1196.0,8.0,-127.0,239.0,2402.0,...,4646.0,2412.0,2340.0,2318.0,2253.0,2194.0,2119.0,3947.0,0,PP
1,2156.0,2139.0,2285.0,1527.0,2052.0,442.0,-223.0,-181.0,243.0,2073.0,...,5061.0,2900.0,1980.0,2032.0,2075.0,1976.0,1976.0,2394.0,1,TT
